CPD ground truth, ws, paper split

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import csv
import sys
import os
import h5py
import simplejson as json

import sys
sys.path.append('../../src')
import data_utils_opp_window_size as data_utils

In [2]:
#load the data
data_path = '/data2/data/zebrahim/OpportunityUCIDataset'

dr = data_utils.data_reader('opportunity', data_path)

reading file 1 of 19
reading file 2 of 19
reading file 3 of 19
reading file 4 of 19
reading file 5 of 19
reading file 6 of 19
reading file 7 of 19
reading file 8 of 19
reading file 9 of 19
reading file 10 of 19
reading file 11 of 19
reading file 12 of 19
reading file 13 of 19
reading file 14 of 19
reading file 15 of 19
reading file 16 of 19
reading file 17 of 19
reading file 18 of 19
reading file 19 of 19
reading file 1 of 1
reading file 1 of 4
reading file 2 of 4
reading file 3 of 4
reading file 4 of 4


In [3]:
n_class = 18 + 1  #all subjects have been enrolled for all 18 activities, last one is for the change point detection. It is multitask ground truth.
def fill_gt(subject_ts):
    t = subject_ts.shape[0]
    gt_ = np.zeros((t, n_class))
    
    for i in range(t):
        gt_[i, subject_ts[i]-1] = 1
        
    idx_change_activity = np.where(subject_ts[:-1] - subject_ts[1:] != 0)[0]
    gt_[idx_change_activity, 18] = 1   
    
    return np.asarray(gt_)

In [14]:
train_gt_box = []
for i in range(len(dr.data['training']['targets'])):
    train_gt_box.append(fill_gt(dr.data['training']['targets'][i]))

In [15]:
validation_gt_box = []
for i in range(len(dr.data['validation']['targets'])):
    validation_gt_box.append(fill_gt(dr.data['validation']['targets'][i]))

In [16]:
test_gt_box = []
for i in range(len(dr.data['test']['targets'])):
    test_gt_box.append(fill_gt(dr.data['test']['targets'][i]))

In [17]:
dr.data['training']['targets']

((32500, 19), (31892, 19))

In [8]:
train_gt_box[1][:,-1].sum()

93.0

In [23]:
dr.data['training']['inputs'][0].shape

(50702, 77)

In [25]:
len(dr.data['training']['inputs'])

19

In [9]:
window_size = 1024
stride = 1024

In [10]:
_, train_data = dr.do_sliding_window(dr.data['training']['inputs'], window_size = window_size, stride = stride, num_dim_expand=0)
_, validation_data = dr.do_sliding_window(dr.data['validation']['inputs'], window_size = window_size, stride = stride, num_dim_expand=0)
_, test_data = dr.do_sliding_window(dr.data['test']['inputs'], window_size = window_size, stride = stride, num_dim_expand=0)

In [11]:
#sliding window of ground truth
_, train_gt = dr.do_sliding_window(train_gt_box, window_size = window_size, stride = stride, num_dim_expand=0)
_, validation_gt = dr.do_sliding_window(validation_gt_box, window_size = window_size, stride = stride, num_dim_expand=0)
_, test_gt = dr.do_sliding_window(test_gt_box, window_size = window_size, stride = stride, num_dim_expand=0)

In [12]:
train_data.shape, validation_data.shape, test_data.shape

((691, 1024, 77), (32, 1024, 77), (119, 1024, 77))

In [22]:
train_gt.shape, validation_gt.shape, test_gt.shape

((691, 1024, 19), (32, 1024, 19), (119, 1024, 19))

In [23]:
data = {}
data['train_data'] = train_data
data['train_gt'] = train_gt

data['validation_data'] = validation_data
data['validation_gt'] = validation_gt


data['test_data'] =  test_data
data['test_gt'] = test_gt

In [25]:
import hickle
path_to_save = '../../data/'
with open(path_to_save+'opp8.hkl', 'w') as fout:
    hickle.dump(data, fout)

In [ ]:

1

In [27]:
len([38, 39, 40, 41, 42, 43, 44, 45, 46, 51, 52, 53, 54, 55, 56, 57, 58, 59,
            64, 65, 66, 67, 68, 69, 70, 71, 72, 77, 78, 79, 80, 81, 82, 83, 84, 85,
            90, 91, 92, 93, 94, 95, 96, 97, 98, 103, 104, 105, 106, 107, 108, 109,
            110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124,
            125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 250])

78

In [28]:
134-38

96

In [34]:
a = np.asarray([item.shape[0]/30/60 for item in dr.data['training']['inputs']])

In [36]:
a.mean()

20.05263157894737